# Исследование надежности заемщиков

Описание проекта   
Заказчик — кредитный отдел банка.   
Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.   
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
# df = pd.read_csv('/home/user-0-1534496771/work/data.csv')
# df.info()

In [2]:
#код ревьюера
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('C:/Users/User/data.csv')

In [3]:
print(df.info())
print()
print('Количество дубликатов:', df.duplicated().sum())
print()
print('Количество пропусков:')
print(df.isna().sum())
print()
print('Минимальные:')
print(df.min())
print()
print('Максимальные:')
print(df.max())
print()
print('Значения количества детей клиента в датасете:')
print(sorted(df['children'].unique()))
print()
print('Значения возраста клиента в датасете:')
print(sorted(df['dob_years'].unique()))
print()
print('Значения образования клиента в датасете:')
print(sorted(df['education'].unique()))
print()
print('Значения семейного положения клиента в датасете:')
print(sorted(df['family_status'].unique()))
print()
print('Значения income_type клиента в датасете:')
print(sorted(df['income_type'].unique()))

print('Значения пола клиента в датасете:')
print(df['gender'].value_counts())
print()
print('Минимальное значение общего трудового стажа в днях среди положительных значений датасета:')
print(min(df[df['days_employed'] > 0]['days_employed']))
print('Максимальное значение общего трудового стажа в днях среди отрицательных значений датасета:')
print(max(abs(df[df['days_employed'] < 0]['days_employed'])))




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None

Количество дубликатов: 54

Количество пропусков:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        217

В результате обзора данных в столбцах days_employed, total_income выявлены пропуски данных, также имеются дубликаты строк, также аномальные (странные) значения: в столбцах days_employed и children встречаются отрицательные значения, в столбце children присутствует значение 20, в столбце days_employed положительные значения от 900 лет (нормыльными значениями представляются отрицательные значения по модулю), в столбце dob_years встречается значение 0, в столбце education встречаются повторы в разных регистрах, в столбце gender присутствует значение XNA.

### Шаг 2.1 Заполнение пропусков

In [4]:
 try:
    print('Количество и доля пропущенных значений в каждом столбце days_employed и\ntotal_income составляют соответственно:', 
          df['days_employed'].isna().sum(), 'шт.', round(df['days_employed'].isna().mean()*100, 1), '%')
    print('Медиана столбца total_income:', round(df['total_income'].median(), 1))
    df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))
except:
    print('Возникла ошибка, нужно разбираться!')

Количество и доля пропущенных значений в каждом столбце days_employed и
total_income составляют соответственно: 2174 шт. 10.1 %
Медиана столбца total_income: 145017.9


В двух столбцах предоставленного датасета отсуствуют значения (NaN) в количестве 2174 шт. в каждом. Это составляет около 10 % данных соответствующих столбцов датасета. Исходя из имеющегося описания даных, пропуски содержатся в столбце days_employed - это общий трудовой стаж в днях и в столбце total_income — это ежемесячный доход. Среди возможных причин появления пропусков - отсуствие занятости заёмщиков, наличие "теневой" занятости заёмщиков, ошибки при выгрузке данных кредитным отделом банка, незаполненность анкетных данных заёмщиками. Заполнение пропусков медианными значениями в данном случае является наилучшим решением, так как целью исследования является анализ поведения клиента по данным, указанным в иных столбцах датасета и заполнение медианными значениями окажет наименьший эффект на результаты исследования.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
try:
    print('Количество аномальных значений столбца days_employed(все положительные значения)',
      df[df['days_employed'] > 0]['days_employed'].count(), ', что составляет долю в размере', round((df[df['days_employed'] > 0]['days_employed'].count()/len(df))*100, 2),
      '%', '''от всего количества записей датасета - исправлено замещением аномальных положительных значений медианами
отрицательных значений по типам занятости клиентов (по модулю соотвествуют описанию данных), отрицательные значения замещены их модулями, оставшиеся заполнены 
медианой всех значений по модулю, отрицательные значения в количестве''', df[df['days_employed'] < 0]['days_employed'].count(),
      'замещены модулем их значений.')
    df['days_employed'] = df[df['days_employed'] < 0].groupby('income_type')['days_employed'].transform("median")
    df['days_employed'] = df['days_employed'].abs()
    df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform("median"))
    #заполняем оставшиеся пропуски, не входящие в группы нормальных значений, медианой колонки
    df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
    print('Количество отрицательных значений столбца children(исправлено - заполнено модулем значений):', df[df['children'] < 0]['children'].count())
    df['children'] = df['children'].abs()
    print('Максимальное значение столбца children(исправлено - заполнено медианой):', df['children'].max())
    df[df['children'] == 20] = df['children'].median()
except:
    print('Возникла ошибка, нужно разбираться!')

Количество аномальных значений столбца days_employed(все положительные значения) 3445 , что составляет долю в размере 16.0 % от всего количества записей датасета - исправлено замещением аномальных положительных значений медианами
отрицательных значений по типам занятости клиентов (по модулю соотвествуют описанию данных), отрицательные значения замещены их модулями, оставшиеся заполнены 
медианой всех значений по модулю, отрицательные значения в количестве 15906 замещены модулем их значений.
Количество отрицательных значений столбца children(исправлено - заполнено модулем значений): 47
Максимальное значение столбца children(исправлено - заполнено медианой): 20


В столбце children содержатся 47 отрицательных значений, что не соотвествует описанию данных. Такое несоотвествие считаем аномалией и заменяем модулем значений.
Максимальное значение столбца children 20, что не соотвествует возможным значениям в соотвествии с описанием данных. Данное несоотвествие считаем аномалией и заменяем медианой значений.

### Шаг 2.3. Изменение типов данных.

In [6]:
df['total_income'] = df['total_income'].astype('int64')

### Шаг 2.4. Удаление дубликатов.

In [7]:
try:
    df = df.drop_duplicates()
    print(df['education'].unique())
    df['education'] = df['education'].str.lower()
    df['family_status'] = df['family_status'].str.lower()
    df['gender'] = df['gender'].str.lower()
    df = df.dropna()
    print(df['family_status'].unique())
    print(df['gender'].unique())
    print(df['purpose'].unique())
    
except:
    print('Возникла ошибка, нужно разбираться!')

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее' 0.0
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
['f' 'm' 'xna']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение 

Выбор метода поиска и удаления дубликатов определяется поставленной задачей. Так для поиска полных дубликатов строк в датасете использован метод drop_duplicates(). Для поиска неполных дубликатов использован метод unique(), т.к. не требуется подсчёт упоминания вариантов каждого значения в датасете. Также использован метод str.lower(), который заменяет значения, переводя в нижний регистр строковые данные. Появление дубликатов обусловлено ошибками при выгрузке данных, ошибками при вводе в всвязи с отсуствием общей категоризации данных на разных участках их ввода.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [8]:
try:
    df1 = df[['education_id', 'education']]
    df2 = df[['family_status_id', 'family_status']]
    df = df.drop(columns={'education', 'family_status'})
except:
    print('Возникла ошибка, нужно разбираться!')

### Шаг 2.6. Категоризация дохода.

In [9]:
def total_income_category(row):
    if row['total_income'] in range(30001):
        return 'E'
    elif row['total_income'] in range(30001, 50001):
        return 'D'
    elif row['total_income'] in range(50001, 200001):
        return 'C'
    elif row['total_income'] in range(200001, 1000001):
        return 'B'
    elif row['total_income'] in range(1000001, (row['total_income']+1)):
        return 'A'
df['total_income_category'] = df.apply(total_income_category, axis = 1)

### Шаг 2.7. Категоризация целей кредита.

In [10]:
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'
try:
    df['purpose_category'] = df['purpose'].apply(purpose_category)
except:
    print('Возникла ошибка, нужно разбираться!')

### Шаг 3. Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [11]:
df_pivot_children = df.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean']).reset_index()
df_pivot_children.columns = df_pivot_children.columns.droplevel(1)
df_pivot_children.columns = ['children', 'total.count', 'debt_1.count', 'debt_1.mean']
df_pivot_children['debt_1.mean'] = (df_pivot_children['debt_1.mean'] * 100).round(2).astype(str) + '%'
df_pivot_children

,children,total.count,debt_1.count,debt_1.mean
0,0.0,14107,1063.0,7.54%
1,1.0,4856,445.0,9.16%
2,2.0,2052,194.0,9.45%
3,3.0,330,27.0,8.18%
4,4.0,41,4.0,9.76%
5,5.0,9,0.0,0.0%


**Вывод 1: Исследуя зависимость между наличием детей у заёмщика и возвратом кредита в срок, можно прийти к выводу, что максимальная доля должников в группах заёмщиков по количеству детей составляют группы с 2-мя и 4-мя детьми. Сравнивая показатели групп заёмщиков с детьми и бездетных заёмщиков, можно прийти к выводу, что заёмщики с детьми чаще допускают просрочку, но незначительно, на порядка 2%. С увеличением количества детей уменьшается репрезентативность выборки. Так для заёмщиков с 5 детьми выборке не имеется случаев посрочки.**

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
df_pivot_family = df.pivot_table(index='family_status_id', values='debt', aggfunc=['count', 'sum', 'mean']).reset_index()
df_pivot_family.columns = df_pivot_family.columns.droplevel(1)
df_pivot_family.columns = ['family_status_id', 'total.count', 'debt_1.count', 'debt_1.mean']
df_pivot_family['debt_1.mean'] = (df_pivot_family['debt_1.mean'] * 100).round(2).astype(str) + '%'
df_pivot_family = df_pivot_family.merge(df2.drop_duplicates().reset_index(drop=True).sort_values('family_status_id'), how='inner', on='family_status_id').drop(columns = 'family_status_id')
df_pivot_family = df_pivot_family[['family_status', 'total.count', 'debt_1.count', 'debt_1.mean']]
df_pivot_family

,family_status,total.count,debt_1.count,debt_1.mean
0,женат / замужем,12295,928.0,7.55%
1,гражданский брак,4151,385.0,9.27%
2,вдовец / вдова,955,63.0,6.6%
3,в разводе,1193,84.0,7.04%
4,не женат / не замужем,2801,273.0,9.75%


**Вывод 2: Исследуя зависимость между семейным положением и просрочками по кредитам, можно прийти к выводу, что большинство просрочек допускают кредиторы относящиеся к группам холостых и состоящих в гражданском браке, менее всех - в разводе и вдовцы(вдовицы), но выборки по ним не репрезентативны.**

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
df_pivot_income = df.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean']).reset_index()
df_pivot_income.columns = df_pivot_income.columns.droplevel(1)
df_pivot_income.columns = ['total_income_category', 'total.count', 'debt_1.count', 'debt_1.mean']
df_pivot_income['debt_1.mean'] = (df_pivot_income['debt_1.mean'] * 100).round(2).astype(str) + '%'
df_pivot_income['debt_1.count'] = df_pivot_income['debt_1.count'].astype(int)
df_pivot_income

,total_income_category,total.count,debt_1.count,debt_1.mean
0,A,25,2,8.0%
1,B,5022,354,7.05%
2,C,15976,1354,8.48%
3,D,350,21,6.0%
4,E,22,2,9.09%


**Вывод 3: Исследуя зависимость между уровнем дохода и просрочками по кредиту, можно прийти к выводу, что максимальная доля просрочек имеется в нерепрезентативной выборке группы с доходом до 30 тыс., тем же недостатоком характеризуется и выборка заёмщиков с доходом от 1 млн. Максимальная просрочка достигается в репрезентативной группе с доходом от 50 тыс. до 200 тыс.**

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [14]:
df_pivot_purpose = df.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean']).reset_index()
df_pivot_purpose.columns = df_pivot_purpose.columns.droplevel(1)
df_pivot_purpose.columns = ['purpose_category', 'total.count', 'debt_1.count', 'debt_1.mean']
df_pivot_purpose['debt_1.mean'] = (df_pivot_purpose['debt_1.mean'] * 100).round(2).astype(str) + '%'
df_pivot_purpose['debt_1.count'] = df_pivot_purpose['debt_1.count'].astype(int)
df_pivot_purpose

,purpose_category,total.count,debt_1.count,debt_1.mean
0,операции с автомобилем,4292,401,9.34%
1,операции с недвижимостью,10778,780,7.24%
2,получение образования,3999,369,9.23%
3,проведение свадьбы,2326,183,7.87%


**Вывод 4: Исследуя зависимость между целями кредита и просрочками по нему, можно прийти к выводу, что больше остальных допускают просрочки заёмщики кредитов на операции с автомобилем и на получение образования. Меньше всех - на операции с недвижимостью, учитывая и самую большую группу заёмщиков с данной целью.**

### Общий вывод:

**Исследование проведено по данным о банковских операциях по обслуживанию клиентов-физических лиц заёмщиков. Данные содержат ряд параметров, позволяющих исследовать потребителькое поведение клиента, а именно причины и факторы просрочек в оплате кредитов. Обобщая результат проведенного исследования можно сделать вывод о том, что существует ряд факторов, указывающих на риск возможной просрочки. Так нами проведён анализ данных, на основании которого установлено, что наиболее подвержены просрочкам по кредитам не женатые/не замужние, а равно состоящие в "гражданском браке" заёмщики, имеющие детей, с уровнем дохода до 30 тыс., а также от 50 тыс. до 200 тыс., пользующиеся кредитом для целей совершения операций с автомобилем или получения образования.**